<a href="https://colab.research.google.com/github/omkar-salunke/accounts/blob/main/both_tally_daybook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# prompt: list of files in the path in python

import os
import pandas as pd
import xml.etree.ElementTree as ET
import pandas as pd
from openpyxl import load_workbook

def extract_element_data(element):
    element_data = {}
    # If the element has no children, return its text
    if len(element) == 0:
        return element.text

    # Otherwise, go deeper into the element's children
    for child in element:
        # Recursively extract data for each child element
        child_data = extract_element_data(child)
        element_data[child.tag] = child_data
    return element_data

# Function to flatten nested dictionaries (e.g., 'amount': {'P': {'T': 100}})
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to recursively search for all 'voucher' elements and extract their data
def find_all_vouchers(element, voucher_tag="VOUCHER"):
    vouchers = []

    # If the current element is a 'voucher', collect its data
    if element.tag == voucher_tag:
        voucher_dict = {}
        # Collect data for all child elements within the voucher
        for sub_child in element:
            extracted_data = extract_element_data(sub_child)
            voucher_dict[sub_child.tag] = extracted_data

        # Flatten the voucher dictionary to handle nested elements
        voucher_dict_flattened = flatten_dict(voucher_dict)
        vouchers.append(voucher_dict_flattened)

    # Recursively search in all children of the current element
    for child in element:
        vouchers.extend(find_all_vouchers(child, voucher_tag))

    return vouchers

# Function to convert the XML to a DataFrame by extracting all vouchers
def xml_to_dataframe_vouchers(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all 'voucher' elements in the XML
    voucher_list = find_all_vouchers(root)

    # Create a DataFrame from the list of voucher dictionaries
    df = pd.DataFrame(voucher_list)
    return df




In [ ]:
# df_bank_match_soe = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='SOE 2024-25')
# df_bank_match_overview_soe = df_bank_match_soe.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

# df_bank_match_me = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='ME 2024-25')
# df_bank_match_overview_me = df_bank_match_me.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

In [3]:
soe_list = '/content/drive/MyDrive/tally_export/SOE/'
me_list = '/content/drive/MyDrive/tally_export/ME/'
files_soe = os.listdir('/content/drive/MyDrive/tally_export/SOE/')
files_me = os.listdir('/content/drive/MyDrive/tally_export/ME/')

In [4]:

# xml_to_dataframe_vouchers('/content/drive/MyDrive/tally_export/ME/DayBook.xml')

In [5]:
req_cols = ['DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.DATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.INSTRUMENTDATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT','NARRATION']
df_raw_collect_soe = pd.DataFrame()
for xt in files_soe:
  path_ct = soe_list
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)
  df_raw_f_ref = pd.DataFrame(df_raw_f)[req_cols]
  df_raw_collect_soe = pd.concat([df_raw_collect_soe, df_raw_f_ref]).reset_index(drop=True)
df_raw_collect_soe
df_raw_soe = df_raw_f_ref.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_soe['date_column'] = pd.to_datetime(df_raw_soe['DATE'], format='%Y%m%d')
df_raw_soe = df_raw_soe.sort_values('date_column')
df_raw_soe['date_column_month'] = df_raw_soe['date_column'].dt.month_name()
df_raw_soe['act_qty'] = df_raw_soe['ALLINVENTORYENTRIES.LIST.ACTUALQTY'].str.extract(r'([\d.]+)').astype(float)
df_raw_soe_sales = df_raw_soe[df_raw_soe['VOUCHERTYPENAME']=='SALES GST']
df_raw_soe_sales_sch = df_raw_soe_sales.groupby(['date_column_month','PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'])['act_qty'].sum()

df_schedule_merge_soe = pd.DataFrame(df_raw_soe_sales_sch).pivot_table(index=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],columns='date_column_month',values='act_qty',aggfunc='sum').reset_index()
# df_schedule = pd.read_excel("/content/drive/MyDrive/tally_export/schedule.xlsx",sheet_name = 'SOE')
# df_schedule_merge_soe = pd.merge(df_schedule,df_raw_soe_sales_schedule,on=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],how='right')
# df_schedule_merge.to_excel(,sheet_name = 'SOE',index=False)
df_raw_soe


,DATE,GUID,VOUCHERTYPENAME,PARTYLEDGERNAME,PARTYGSTIN,CLASSNAME,PARTYNAME,CONSIGNEEGSTIN,PERSISTEDVIEW,ALLINVENTORYENTRIES.LIST.STOCKITEMNAME,...,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT,NARRATION,date_column,date_column_month,act_qty
0,20250401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c092,SALES GST,AKAR INDUSTRY,27AACFA4208J1ZP,SALES 18%,AKAR INDUSTRY,27AACFA4208J1ZP,Invoice Voucher View,DIFF.CASE LOCK PLATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,13450.0
8,20250401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c09f,Purchase,NOBLE AIR GASES,27AAQFN1807Q1ZL,NaN,NOBLE AIR GASES,27BCHPS7928D1Z8,Invoice Voucher View,Liquid Co2 (30kg),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,60.0
7,20250401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c09e,Purchase,AGRIYA CASTING PVT LTD,27AAHCA2476Q1Z2,PURCHASE GST@18%,AGRIYA CASTING PVT LTD,27BCHPS7928D1Z8,Invoice Voucher View,TRANSPORTATION CHARGES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,NaN
6,20250401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c098,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,SALES 18%,V.S.AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,Invoice Voucher View,FLANGE VTU SIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,1000.0
5,20250401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c097,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,SALES 18%,V.S.AUTO TECH PRIVATE LIMITED,27AADCV3787B1Z8,Invoice Voucher View,FLANGE QUADRANT SIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01,April,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,20250514,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c1da,SALES GST,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,SALES 18%,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,Invoice Voucher View,RETURN SPRING BKT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-14,May,670.0
251,20250514,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c1db,SALES GST,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,SALES 18%,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,Invoice Voucher View,LOCKING PLATE PART.NO.322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-14,May,2193.0
252,20250514,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c1dc,SALES GST,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,SALES 18%,SAVITA AUTO INDUSTRIES (N),27APAPK0624D1ZI,Invoice Voucher View,HEAD LAMP BKT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-14,May,500.0
253,20250514,c00e677e-43c1-4bea-8147-e2e4905eacab-0000c1dd,SALES GST,PATIL-N-PATIL ENGINEERING,27AAVPP3274C1ZD,SALES 28%,PATIL-N-PATIL ENGINEERING,27AAVPP3274C1ZD,Invoice Voucher View,24165221 BKT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-14,May,1540.0


In [6]:
req_cols = ['DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.DATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.INSTRUMENTDATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT','NARRATION']
df_raw_collect_soe = pd.DataFrame()
for xt in files_me:
  path_ct = me_list
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)
  df_raw_f_ref = pd.DataFrame(df_raw_f)[req_cols]
  df_raw_collect_soe = pd.concat([df_raw_collect_soe, df_raw_f_ref]).reset_index(drop=True)
df_raw_collect_soe
df_raw_soe = df_raw_f_ref.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_soe['date_column'] = pd.to_datetime(df_raw_soe['DATE'], format='%Y%m%d')
df_raw_soe = df_raw_soe.sort_values('date_column')
df_raw_soe['date_column_month'] = df_raw_soe['date_column'].dt.month_name()
df_raw_soe['act_qty'] = df_raw_soe['ALLINVENTORYENTRIES.LIST.ACTUALQTY'].str.extract(r'([\d.]+)').astype(float)
df_raw_soe
df_raw_soe_sales = df_raw_soe[df_raw_soe['VOUCHERTYPENAME']=='Sales Gst']
df_raw_soe_sales_sch = df_raw_soe_sales.groupby(['date_column_month','PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'])['act_qty'].sum()

df_schedule_merge_me = pd.DataFrame(df_raw_soe_sales_sch).pivot_table(index=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],columns='date_column_month',values='act_qty',aggfunc='sum').reset_index()
# df_schedule = pd.read_excel("/content/drive/MyDrive/tally_export/schedule.xlsx",sheet_name = 'ME')
# df_raw_soe_sales_schedule
# df_schedule_merge_me = pd.merge(df_schedule,df_raw_soe_sales_schedule,on=['PARTYLEDGERNAME','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME'],how='right')

# writer = pd.ExcelWriter("/content/drive/MyDrive/tally_export/schedule_overview.xlsx", engine = 'openpyxl')
# df_schedule_merge_soe.to_excel(writer, sheet_name = 'SOE')
# df_schedule_merge_me.to_excel(writer, sheet_name = 'ME')
# writer.close()


In [17]:
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.utils.dataframe import dataframe_to_rows

# file_path = "/content/drive/MyDrive/tally_export/schedule_overview.xlsx"

# # Step 1: Load existing workbook
# wb = load_workbook(filename=file_path)

# # Step 2: Remove and recreate target sheets
# for sheet_name in ['SOE', 'ME']:
#     if sheet_name in wb.sheetnames:
#         del wb[sheet_name]
#     wb.create_sheet(title=sheet_name)

# # Step 3: Write new data to SOE
# ws_soe = wb['SOE']
# for r in dataframe_to_rows(df_schedule_merge_soe, index=False, header=True):
#     ws_soe.append(r)

# # Step 4: Write new data to ME
# ws_me = wb['ME']
# for r in dataframe_to_rows(df_schedule_merge_me, index=False, header=True):
#     ws_me.append(r)

# # Step 5: Save to a **new file first** to avoid Google Drive locking issue
# temp_path = "/content/drive/MyDrive/temp_schedule_overview.xlsx"
# wb.save(temp_path)

# # Step 6: Overwrite the original file safely
# import shutil
# shutil.copy(temp_path, file_path)


'/content/drive/MyDrive/tally_export/schedule_overview.xlsx'

In [19]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

file_path = "/content/drive/MyDrive/tally_export/schedule_overview.xlsx"

# Load existing workbook
wb = load_workbook(filename=file_path)

# Define a function to clear and update a sheet
def update_sheet(sheet_name, df):
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        # Clear existing cells (except formats)
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, max_col=ws.max_column):
            for cell in row:
                cell.value = None
    else:
        ws = wb.create_sheet(title=sheet_name)

    # Write new data
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

# Update only the target sheets
update_sheet('SOE', df_schedule_merge_soe)
update_sheet('ME', df_schedule_merge_me)

# Save to a temp file then replace the original (for GDrive safety)
temp_path = "/content/temp_schedule_overview.xlsx"
wb.save(temp_path)

import shutil
shutil.copy(temp_path, file_path)


'/content/drive/MyDrive/tally_export/schedule_overview.xlsx'

In [ ]:


df_raw_collect_me = pd.DataFrame()
for xt in files_me:
  path_ct = me_list
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)[req_cols].reset_index(drop=True)
  df_raw_f_ref = pd.DataFrame(df_raw_f)
  df_raw_collect_me = pd.concat([df_raw_collect_me, df_raw_f_ref]).reset_index(drop=True)

df_raw_me = df_raw_collect_me.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_me['date_column'] = pd.to_datetime(df_raw_me['DATE'], format='%Y%m%d')
df_raw_me = df_raw_me.sort_values('date_column')
df_raw_me

,DATE,GUID,VOUCHERTYPENAME,PARTYLEDGERNAME,PARTYGSTIN,CLASSNAME,PARTYNAME,CONSIGNEEGSTIN,PERSISTEDVIEW,ALLINVENTORYENTRIES.LIST.STOCKITEMNAME,...,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT,NARRATION,date_column
0,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001995,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,HORN MTG BKT P.NO.007655727V91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01
13,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-00001aca,Contra,CENTRAL BANK OF INDIA 7269,NaN,NaN,NaN,NaN,Accounting Voucher View,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WITHDRAWAL TRANSFER\r\n,2025-04-01
12,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-000019a6,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,BKT ASSLY AIR CLEANER MTG P.NO.008019722 V11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01
11,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-000019a5,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,RADIATOR TOP SEALING ASSLY WELDING PA.NO.00765...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01
10,20250401,9560e508-101c-4d25-b11f-69c1f97a4f64-000019a4,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,PANEL FRONT FLOOR REAR LH/RH PART NO.007650723...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,20250508,9560e508-101c-4d25-b11f-69c1f97a4f64-00001b00,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,PANEL FLOOR FRONT SPT BKT LH P.NO.007650686V01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-08
275,20250508,9560e508-101c-4d25-b11f-69c1f97a4f64-00001aff,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,BKT ASSLY AIR CLEANER MTG P.NO.008019722 V11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-08
274,20250508,9560e508-101c-4d25-b11f-69c1f97a4f64-00001afe,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,RADIATOR TOP SEALING ASSLY WELDING PA.NO.00765...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-08
277,20250508,9560e508-101c-4d25-b11f-69c1f97a4f64-00001b01,Sales Gst,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,SALES GST 18%,SHREE SAI ENGINEERING,27AAOPR0585P1ZR,Invoice Voucher View,PANEL FLOOR FRONT SPT BKT RH P.NO.007650687V01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-08


In [ ]:
def add_req_fields(df):
  df['party_name'] = df[['PARTYNAME', 'ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING', 'ALLLEDGERENTRIES.LIST.LEDGERNAME']].bfill(axis=1).iloc[:, 0]
  df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
  df['tax_out_2'] = df[['LEDGERENTRIES.LIST.VATEXPAMOUNT']].bfill(axis=1).iloc[:, 0]

  # df[''] = df[''].astype(float)
  df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'] = df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'].astype(float)
  df['tax_out_1'] = df['tax_out_1'].astype(float)
  df['tax_out_2'] = df['tax_out_2'].astype(float)

  df['amount_coalesce'] = df[['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT', 'ALLLEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
  df['amount_coalesce'] = df['amount_coalesce'].astype(float)
  df['tax_coalesce'] = df[['tax_out_1', 'tax_out_2']].sum(axis=1)
  df['total_coalesce'] = df[['tax_coalesce', 'amount_coalesce']].sum(axis=1)

  df = df[['party_name','total_coalesce','amount_coalesce','tax_coalesce','tax_out_1','tax_out_2']+req_cols]
  return df

In [ ]:
df_me = add_req_fields(df_raw_me)
df_me['DATE'] = pd.to_datetime(df_me['DATE'])
df_me['month_of_day'] =df_me['DATE'].dt.month
df_me['quarter_of_day'] =df_me['DATE'].dt.quarter

df_soe = add_req_fields(df_raw_soe)
df_soe['DATE'] = pd.to_datetime(df_soe['DATE'])
df_soe['month_of_day'] =df_soe['DATE'].dt.month
df_soe['quarter_of_day'] =df_soe['DATE'].dt.quarter


<ipython-input-80-bc721fa37c88>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['amount_coalesce'] = df[['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT', 'ALLLEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
<ipython-input-81-9f1d164c3bb6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me['DATE'] = pd.to_datetime(df_me['DATE'])
<ipython-input-81-9f1d164c3bb6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [ ]:

df_soe['total_coalesce'] = df_soe['total_coalesce'].astype('float')
df_summary_all_soe = pd.pivot_table(df_soe, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
df_summary_all_soe['balance'] = 0
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='TIRUMALA MULTI TECHNOLOGIES'].index,'balance'] = 467646
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SAVITA AUTO INDUSTRIES (N)'].index,'balance'] = 195853
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='PATIL-N-PATIL ENGINEERING	'].index,'balance'] = 55660
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='NANDAMURI TECHNOS'].index,'balance'] = 35255
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='V.S.AUTO TECH PRIVATE LIMITED'].index,'balance'] = 4699956
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='A.B.ENGINEERING WORKS'].index,'balance'] = 11598

df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='AKAR INDUSTRY'].index,'balance'] = 317899

df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SURYA AUTO INDUSTRIES'].index,'balance'] = 23165


df_summary_all_soe = df_summary_all_soe.fillna(0)
df_summary_all_soe['total_due']=df_summary_all_soe[['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note',
       'Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance']].sum(axis=1)

df_me['total_coalesce'] = df_me['total_coalesce'].astype('float')
df_summary_all_me = pd.pivot_table(df_me, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
df_summary_all_me['balance'] = 0
df_summary_all_me = df_summary_all_me.fillna(0)
df_summary_all_me['total_due']=df_summary_all_me[[ 'Contra','Delivery Note','Debit Note','Sales Gst', 'Journal', 'Payment', 'Purchase', 'Receipt','Sales Gst','balance']].sum(axis=1)

<ipython-input-99-c45c9a62cf25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_soe['total_coalesce'] = df_soe['total_coalesce'].astype('float')
<ipython-input-99-c45c9a62cf25>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me['total_coalesce'] = df_me['total_coalesce'].astype('float')


In [ ]:
# terms = ['SHREE SAI ENGINEERING', 'XYZ COMPANY'] # Function to match and replace terms def match_and_replace(transaction): for term in terms: if term.replace(" ", "").lower() in transaction.replace(" ", "").lower(): return term return transaction # Or return a default value if no match is found # Apply function to the column df['renamed_column'] = df['transaction_column'

In [ ]:
# df_bank_match
df_summary_all_bank_soe = pd.merge(df_summary_all_soe,df_bank_match_overview_soe,on=['party_name'],how='outer')
df_summary_all_bank_soe = df_summary_all_bank_soe.sort_values(by='SALES GST', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='SALES GST', ascending=False).reset_index(drop=True)

# df_bank_match
df_summary_all_bank_me = pd.merge(df_summary_all_me,df_bank_match_overview_me,on=['party_name'],how='outer')
df_summary_all_bank_me = df_summary_all_bank_me.sort_values(by='Sales Gst', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='Sales Gst', ascending=False).reset_index(drop=True)



In [ ]:

df_sales = df_soe[df_soe['VOUCHERTYPENAME']=='SALES GST'].reset_index(drop=True)
# df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
df_sales_g = df_sales.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"total_sales","PARTYLEDGERNAME":"party_name"})
date_45d = max(df_sales['DATE']) - pd.Timedelta(days=45)
due45 = df_sales[df_sales['DATE'] > date_45d]
due45_g = due45.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_45d","PARTYLEDGERNAME":"party_name"})

date_60d = max(df_sales['DATE']) - pd.Timedelta(days=60)
due60 = df_sales[df_sales['DATE'] > date_60d]
due60_g = due60.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_60d","PARTYLEDGERNAME":"party_name"})

date_90d = max(df_sales['DATE']) - pd.Timedelta(days=90)
due90 = df_sales[df_sales['DATE'] > date_90d]
due90_g = due90.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_90d","PARTYLEDGERNAME":"party_name"})

# df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe,df_sales_g,on=['party_name'],how='left')
df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe,due45_g,on=['party_name'],how='left')
df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe_due,due60_g,on=['party_name'],how='left')
df_summary_all_bank_soe_due = pd.merge(df_summary_all_bank_soe_due,due90_g,on=['party_name'],how='left')
df_summary_all_bank_soe_due = df_summary_all_bank_soe_due.fillna(0)
df_summary_all_bank_soe_due['true_credit'] = df_summary_all_bank_soe_due['Credit']-df_summary_all_bank_soe_due['balance']
# df_summary_all_bank_soe_due

In [ ]:

df_sales = df_me[df_me['VOUCHERTYPENAME']=='Sales Gst'].reset_index(drop=True)
# df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
df_sales_g = df_sales.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"total_sales","PARTYLEDGERNAME":"party_name"})
date_45d = max(df_sales['DATE']) - pd.Timedelta(days=45)
due45 = df_sales[df_sales['DATE'] > date_45d]
due45_g = due45.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_45d","PARTYLEDGERNAME":"party_name"})

date_60d = max(df_sales['DATE']) - pd.Timedelta(days=60)
due60 = df_sales[df_sales['DATE'] > date_60d]
due60_g = due60.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_60d","PARTYLEDGERNAME":"party_name"})

date_90d = max(df_sales['DATE']) - pd.Timedelta(days=90)
due90 = df_sales[df_sales['DATE'] > date_90d]
due90_g = due90.groupby(['PARTYLEDGERNAME'])['total_coalesce'].sum().reset_index().rename(columns={"total_coalesce":"Sale_90d","PARTYLEDGERNAME":"party_name"})

# df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me,df_sales_g,on=['party_name'],how='left')
df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me,due45_g,on=['party_name'],how='left')
df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me_due,due60_g,on=['party_name'],how='left')
df_summary_all_bank_me_due = pd.merge(df_summary_all_bank_me_due,due90_g,on=['party_name'],how='left')
df_summary_all_bank_me_due = df_summary_all_bank_me_due.fillna(0)
df_summary_all_bank_me_due['true_credit'] = df_summary_all_bank_me_due['Credit']-df_summary_all_bank_me_due['balance']
# df_summary_all_bank_me_due

In [ ]:
import numpy as np
columns_to_round_soe = ['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note','Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance','total_due', 'Debit', 'Credit','Sale_45d','Sale_60d','Sale_90d']
df_summary_all_bank_soe_due[columns_to_round_soe] = df_summary_all_bank_soe_due[columns_to_round_soe].apply(np.ceil)

columns_to_round_me = ['Contra', 'Debit Note', 'Delivery Note', 'Journal','Payment', 'Purchase', 'Receipt', 'Sales Gst', 'balance', 'total_due','Debit', 'Credit','Sale_45d','Sale_60d','Sale_90d']
df_summary_all_bank_me_due[columns_to_round_me] = df_summary_all_bank_me_due[columns_to_round_me].apply(np.ceil)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime

# Define file path and sheet name
file_path = "/content/drive/My Drive/2024_25/daybook.xlsx"
original_sheet_name = "SOE_overview"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_summary_all_bank_soe_due.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [ ]:
original_sheet_name = "SOE_raw_xml"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_soe.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [ ]:
original_sheet_name = "ME_overview"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_summary_all_bank_me_due.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [ ]:
original_sheet_name = "ME_raw_xml"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_me.to_excel(writer, sheet_name=original_sheet_name, index=False)

In [ ]:
original_sheet_name = "SOE24-25"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_bank_match_soe.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [ ]:
original_sheet_name = "ME24-25"

# Get the current date to use in the renamed sheet name
date_str = datetime.now().strftime("%Y_%m_%d")
renamed_sheet_name = f"{original_sheet_name}_{date_str}"

# Load the workbook and rename the sheet if it exists
workbook = load_workbook(file_path)

if original_sheet_name in workbook.sheetnames:
    # Rename the existing sheet
    sheet = workbook[original_sheet_name]
    sheet.title = renamed_sheet_name
    sheet.sheet_state = 'hidden'  # Hide the sheet

# Save the workbook after renaming and hiding the sheet
workbook.save(file_path)

# Write the new data to the original sheet name
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    df_bank_match_me.to_excel(writer, sheet_name=original_sheet_name, index=False)


In [ ]:
# df_summary_all_bank_soe_due.to_excel("/content/drive/My Drive/2024_25/daybook.xlsx",sheet_name='SOE_overview')

# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_soe.to_excel(writer,sheet_name='SOE_raw_xml')
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_summary_all_bank_me_due.to_excel(writer,sheet_name='ME_overview')
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
#   df_me.to_excel(writer,sheet_name='ME_raw_xml')

In [ ]:
df_soe

,party_name,total_coalesce,amount_coalesce,tax_coalesce,tax_out_1,tax_out_2,DATE,GUID,VOUCHERTYPENAME,PARTYLEDGERNAME,...,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT,NARRATION,month_of_day,quarter_of_day
0,AKAR INDUSTRY,44765.42,37936.8,6828.62,3414.31,3414.31,2024-04-01,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b597,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
1,AKAR INDUSTRY,36464.84,30902.4,5562.44,2781.22,2781.22,2024-04-01,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b598,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
2,AKAR INDUSTRY,23267.84,18178.0,5089.84,2544.92,2544.92,2024-04-01,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b599,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
3,V.S.AUTO TECH PRIVATE LIMITED,186487.20,158040.0,28447.20,14223.60,14223.60,2024-04-01,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b59a,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
4,V.S.AUTO TECH PRIVATE LIMITED,54257.58,45981.0,8276.58,4138.29,4138.29,2024-04-01,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b59b,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,AKAR INDUSTRY,19107.16,16192.5,2914.66,1457.33,1457.33,2024-10-30,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc36,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,4
1303,SAVITA AUTO INDUSTRIES (N),74340.00,63000.0,11340.00,5670.00,5670.00,2024-10-30,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc31,SALES GST,SAVITA AUTO INDUSTRIES (N),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,4
1302,PATIL-N-PATIL ENGINEERING,34220.16,21936.0,12284.16,6142.08,6142.08,2024-10-30,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc30,SALES GST,PATIL-N-PATIL ENGINEERING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,4
1304,SAVITA AUTO INDUSTRIES (N),19700.10,16695.0,3005.10,1502.55,1502.55,2024-10-30,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc32,SALES GST,SAVITA AUTO INDUSTRIES (N),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,4
